# TD 2

***

*Blockchains as a data structure*

***

#### Blockchain and Applications Project

As part of the evaluation for this course on Blockchain and Applications, over the next four tutorial sessions, you will **code your own blockchain**, including **Smart Contracts**. The aim of this tutorial and the next three is to *guide you step by step* in its creation.

Each of the four tutorials dedicated to the project involves **implementing a set of features** that will allow you to run a **real blockchain on your computer** by the end.

This **first tutorial** will involve the implementation of the **data structure**. It will build on all the concepts covered in the course.

Throughout the tutorial, we will intentionally make all the fields of the blockchain public, because anyway, if they are modified, it will invalidate the rest of the blockchain.

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


At the same time as this tutorial, you should have downloaded a folder named "**helpers**". It contains two important scripts that cover the implementation details of certain cryptographic functions useful for our course. These functions rely on the ```pycryptodome``` package that you installed during Tutorial 1. You do not need to read their content, but you are welcome to do so if you are interested.

To summarize, and to make sure that you have the correct architecture for all the tutorials, here is how the different folders are organized:

* Assingments/
    * scripts/
        * helpers/
            * \_\_init\_\_.py
            * cryptography_base.py
            * rsa_cryptography.py
        * *You'll put all of your scripts here*
    * notebooks/
        * **This notebook (Assignment_2)**
        * ...
    * ...

***To validate this architecture, the following cell should run without error and display a beautiful hash.***

In [7]:
from sys import path

path.append('../scripts')           # We add the "scripts" folder to our search path

from helpers import cryptography

cryptography.hash_string('JUNIA')   # A RSA hash function

ModuleNotFoundError: No module named 'Crypto'

The small module `cryptography` that I have implemented for you contains all the **cryptographic functions** you will need during the project. To see the **list of functions** available in this module, you can have them displayed **in Jupyter Notebook by pressing** `Tab` **after typing** `cryptography.` *(don't forget the period)*.

> The names of the functions are clear enough to allow you to differentiate them easily. Similarly, the order of the function arguments is reflected in the name of the function. For example, the function `has_public_key_signed_this_hash` has the signature `cryptography.has_public_key_signed_this_hash(publicKey, signature, hash)`.

***

In addition to the cryptography module, you have a small `timestamp` module, useful for dating.

In [135]:
from helpers import timestamp

timestamp.now()  # Time in millisecondes since an arbitrary date

1736945826941

To ensure a good understanding of the rest of the tutorial, here is a list of elements covered in the course that we will use, along with their **variable type** and an example:

* **Public Key** (`publicKey`): `string(588)` (e.g., <font color="bb9534">"5329b3ede6de58cf..."</font>)
* **Private Key** (`privateKey`): `string(2380)` (e.g., <font color="bb9534">"f5b38ccd9dc77221..."</font>)
* **Creation Date** (`timestamp`): `int` in **milliseconds** (e.g., <font color="2267a5">1673183195004</font>)
* **Signature** (`signature`): `string(512)` (e.g., <font color="bb9534">"2c290ba99465f4b5..."</font>)
* **Block Number in the Blockchain** (`indexInBlockchain`): `int` (e.g., <font color="2267a5">1</font>)
* **Parent Block (previous block in the blockchain)** (`parentBlockHash`): `string(64)` (e.g., <font color="bb9534">"22b46d2c579aa2c2..."</font>)

Generally, all our hashes will have a length of 64.

***

**Throughout this tutorial and the next three, you will mostly have coding tasks with hints available to reveal by yourself if needed. The order indicates their level of help. Try to use them only if you are really stuck.**

***

### Certificate

We choose to call "**certificate**" an **arbitrary entry in the registry** of a block. It is this famous document we saw in class that is issued and signed by a single person. A certificate (`Certificate`) therefore *necessarily* contains an **issuer** (`issuerPublicKey`) and a **signature** (`signature`). Conceptually, a certificate does not need to contain data, as this depends on the type of certificate.

To reinforce the conformity of a certificate, we also add its **creation date** (`timestamp`), which we choose to express in **milliseconds** for easier sorting (number of milliseconds since an arbitrary date).

***

<font color="8888CC">Create a file `certificate.py` in the "scripts" folder.</font>

<font color="8888CC">In this file, you must create a `Certificate` class containing a constructor and public fields in accordance with the statement just above.<br/><br/> This class will serve as the base class for any registry entry you will characterize in the future (you can then build classes that inherit from `Certificate` if they are meant to be signed and stored in the blockchain).</font>

<font color="8888CC">The `signature` field just needs to exist, regardless of the value with which you initialize it.</font>

> <details><summary><strong>Click to display help 1</strong></summary>Use the <code>timestamp</code> module that I introduced above.</details>

> <details><summary><strong>Click to display help 2</strong></summary>Your constructor must take the issuer's public key <code>issuerPublicKey</code> as a parameter and initialize the fields <code>issuerPublicKey</code>, <code>timestamp</code>, and <code>signature</code>.</details>

In [136]:
from certificate import Certificate

dummyCertificate = Certificate('9cbe2a6713')

assert dummyCertificate.issuerPublicKey == '9cbe2a6713'         # The issuer is "9cbe2a6713"
assert 'signature' in dummyCertificate.__dict__                 # The "signature" field exists
assert dummyCertificate.timestamp                               # Date is not empty

'Success!'

'Success!'

<font color="8888CC">Create a `build_payload()` function in this new class that returns the certificate data that are part of its **payload**.

You have the freedom to choose the return format of your function (list, dictionary, etc.) as long as it contains all the data of the payload.</font>

> <details><summary><strong>Click to display help 1</strong></summary>As a reminder: the payload of a certificate corresponds to the certificate data that are considered unalterable. It represents the set of values of the document that we want to protect and that characterize the certificate. Not all the values of the certificate necessarily form part of the payload.</details>

> <details><summary><strong>Click to display help 2</strong></summary>You could, for example, return a Python dictionary where the keys are the names of the fields that are part of the payload, and the values are the values of these fields.</details>

> <details><summary><strong>Click to display help 3</strong></summary>If you include the signature in the payload, won't you have a problem if you sign the certificate twice?</details>

In [137]:
dummyCertificate.build_payload()

{'issuerPublicKey': '9cbe2a6713', 'timestamp': 1736945826951}

<font color="8888CC">Using this new function `build_payload()`, now introduce the function `hash()` which transforms the certificate into a hexadecimal `string` hash.

Here too, you have the freedom to choose the transformation process. The main thing is that the hash should always be the same for the same payload, and vary if the payload varies.</font>

> You are entirely allowed to use the built-in `str` function, even though it's not very secure, as for example `str(1.0)` and `str(1.0000000000000001)` both give `'1.0'`. `str` is a function highly dependent on the implementation of the object you are converting. Ideally, you should go through the binary representation of the objects. But I consider that to be beyond the scope of this tutorial.

> <details><summary><strong>Click to display help 1</strong></summary>You need to write a function that starts from <code>build_payload()</code>, transforms this payload into a <code>string</code> in the way you wish, and then changes the <code>string</code> into a hash using the <code>cryptography</code> module.</details>

> <details><summary><strong>Click to display help 2</strong></summary>If your <code>build_payload()</code> function returns a list, you can use the <code>dumps(object)</code> function from the <code>json</code> package.<br/>If it returns a dictionary, you can use it too but add the <code>sort_keys=True</code> parameter, because from one Python version to another, your elements in the dictionary will be in a different order.</details>

In [138]:
dummyCertificate.hash()

'5ce4c419b4b30411f036e1ae7eb5c4a35d6d1dff99e1ce8527b65bee7fb5e108'

To help you understand how impossible it is to find the same hash twice, even if you only slightly modify the data, observe the result of the following cell.

In [139]:
dummyCertificate.timestamp -= 1            # Here we only change one bit of the whole certificate
dummyCertificate.hash()

'f114455fdcdc3f280dc67bcb1baa1b4b746db3c426b06bd928c6451d1a7448eb'

With this `hash()` function, we now have the first of the two pieces of the puzzle to be able to cryptographically sign our certificates. The missing piece is the **private key / public key** pair.

Before moving on to this missing piece, let's implement one last function that will prove very useful later on (especially for the end of the tutorial...)

***

<font color="8888CC">Create the function `equals(otherCertificate)` that returns `True` if the certificate and `otherCertificate` are identical, and `False` otherwise.
    
Consider what really makes two certificates identical. You will not need to implement this function in classes that inherit from `Certificate`.</font>

> <details><summary><strong>Click to display help 1</strong></summary>No need to make it complicated, there's a reason this is the function you're implementing last in this class.</details>

> <details><summary><strong>Click to display help 2</strong></summary>Use the `hash()` function.</details>

In [140]:
assert dummyCertificate.equals(dummyCertificate)                    # A certificate is equal to itself

otherCertificate = Certificate(dummyCertificate.issuerPublicKey)
otherCertificate.timestamp = dummyCertificate.timestamp
assert otherCertificate.equals(dummyCertificate)                    # Two certificates having different signatures

otherCertificate.issuerPublicKey = 'bbc887326a'
otherCertificate.timestamp = 84539
assert not otherCertificate.equals(dummyCertificate)                # Two completely different certificates

"Success!"

'Success!'

### Wallet

In the course, we covered how a **private key / public key pair** works, and what a wallet is, especially in the realm of cryptocurrencies. To simplify the terminology, we will keep the term "**wallet**" to refer to this **pair of keys**.

Thus, the sole and unique role of our wallet will be to **store this pair of keys** and to **sign** using it.

***

<font color="8888CC">Create a file `wallet.py` containing the `Wallet` class. This class should contain a private key and a public key upon creation. The public key is represented by the field `publicKey`.
    
**To stay true to the aspect of secrecy, ensure that the private key is as inaccessible as possible from the outside.** However, the public key should be accessible.</font>

> The `cryptography` module can help you generate a pair of keys.

> In Python, putting two underscores `__` at the beginning of the field name allows very restricted access to the said field. Only the internal code of the class can then access it (if one stays within a classic development framework, of course, this limit can be easily crossed with a bit more knowledge in Python...).

In [141]:
from wallet import Wallet

dummyWallet = Wallet()

assert len(dummyWallet.publicKey) == 588           # We expect a RSA public key of length 588

dummyWallet.publicKey

'30820122300d06092a864886f70d01010105000382010f003082010a0282010100b5af16d1c5ce0f9631f3c09254da4685b529d63cfe45f8b8782478f6c817760186b80762a880f5b5e5c6a5e5ab7b2175398915b0ff0672b650515485d4837e8d6b169e6af691ba9b617ede3fdd9255151405b2f8801f1cca8aa057d50655f5a87f615542f8456356d4c525219f0421911eae9c6ed41c02b64f13235fef617a61ad1417c2aa13e16754e4b93e63d99244c24c1741ff4ff19c3f80dee50b92114845cd1f8f744fa2956b0cf4f3da422ae3fdc7ab31c66a8591f73535b6e5e7d0264f847f76839c2d43aefec4e1a34f20c5ecfdc1710df73a020d7ac1c699b9de7eca415de0d144eb81fe9e88c23425229bf3eca99512cddb613a39b3dac5cb56dd0203010001'

We only have to use our **private key** to **sign any certificate** now.

***

<font color="8888CC">Add to the `Wallet` class a function `sign(certificate)` that takes a certificate as input and signs it. **This function does not return anything: it simply applies the signature to the certificate.**</font>

> <details><summary><strong>Click to display help 1</strong></summary>You have a great function in the `cryptography` module that signs any <code>string</code>.</details>

> <details><summary><strong>Click to display help 2</strong></summary>We could have implemented <code>Wallet</code> first, but to sign a certificate you first need to transform it into a <code>string</code>, something you can now do thanks to your work...</details>

In [142]:
dummyWallet.sign(dummyCertificate)

assert dummyCertificate.signature                          # We apply the signature

previousSignature = dummyCertificate.signature

dummyWallet.sign(dummyCertificate)

assert previousSignature == dummyCertificate.signature     # If we sign twice, the same signature should be applied

'Success!'

'Success!'

### Block

We learned during the course that a **block** is a **set of data**, it comes **from an issuer** (the forger), and this issuer must **sign** it. Does this remind you of something?

*A block is nothing more than a certificate that contains other certificates.*

However, you need to add additional fields related to the use of blocks: `indexInBlockchain`, which keeps track of the block number in the blockchain, `parentBlockHash`, which refers to the block just before it in the chain (via its hash), and finally `certificateList`, which is a list containing all the certificates stored in the block (the block's data).

***

<font color="8888CC">Following this schema, create the file `block.py` containing the `Block` class. **This class must inherit from the** `Certificate` **class, and its constructor then takes 4 parameters.**</font>

> <details><summary><strong>Click to display help 1</strong></summary>The built-in function <code>super()</code> returns <code>self</code> but with the interface of the parent class (here the interface of <code>Certificate</code>). You can therefore reuse functions that you have written in the parent class, which is very useful when you want to replace them while still using them.</details>

> <details><summary><strong>Click to display help 2</strong></summary>In the constructor, do not forget to call <code>super().__init__(issuerPublicKey)</code>.</details>

In [143]:
from block import Block

dummyCertificateList = [dummyCertificate, otherCertificate]
dummyBlock = Block('9cbe2a6713', 0, 'void', dummyCertificateList)

assert dummyBlock.__class__.__bases__[-1].__name__ == "Certificate" # Our block is a certificate
assert dummyBlock.issuerPublicKey == '9cbe2a6713'                   # The issuer is "9cbe2a6713"
assert dummyBlock.indexInBlockchain == 0                            # The index is 0
assert dummyBlock.parentBlockHash == 'void'                         # The parent is the block of hash "void" 
assert len(dummyBlock.certificateList) == 2                         # The block contains 2 certificates

'Success!'

'Success!'

<font color="222277">Still in the `Block` class, rewrite (override) the `build_payload()` function to include the block's data.</font>

> <details><summary><strong>Click to display help 1</strong></summary>Be careful to have in the block's payload the payloads of its certificates and not the certificates directly.</details>

> <details><summary><strong>Click to display help 2</strong></summary>This function already exists in `Certificate`, so you need to redefine it, but inside it, you must also call the base function by writing `super().build_payload()`</details>

In [144]:
dummyBlock.build_payload()

{'issuerPublicKey': '9cbe2a6713',
 'timestamp': 1736945827132,
 'indexInBlockchain': 0,
 'parentBlockHash': 'void',
 'certificateList': [{'issuerPublicKey': '9cbe2a6713',
   'timestamp': 1736945826950},
  {'issuerPublicKey': 'bbc887326a', 'timestamp': 84539}]}

You don't need to rewrite the `hash()` function since it is based exclusively on the `build_payload()` function which you have already completed.

In [145]:
dummyBlock.hash()

'9f1532c3d41a9381d45e94ebef315eaed1cbdd2bf655474bdac105d22eb24a56'

### Blockchain

We have implemented blocks and certificates, and thanks to our wallet, we are capable of signing all of this. We now need to create our **blockchain** so that we can add our freshly generated blocks to it.

***

<font color="8888CC">Create the file `blockchain.py`. In this file, implement the `Blockchain` class, with a public field `blockList` that contains the list of blocks in their index order and the appropriate constructor.

Remember that a blockchain is **never empty**; it always has at least one specific block we talked about in class...</font>

> <details><summary><strong>Click to display help 1</strong></summary>Initialize <code>blockList</code> with the genesis block.</details>

> <details><summary><strong>Click to display help 2</strong></summary>The genesis block can contain anything, as long as it's always the same from one blockchain to another.</details>

In [146]:
from blockchain import Blockchain

dummyBlockchain = Blockchain()

assert len(dummyBlockchain.blockList) == 1    # I expected 1 block here, which we have seen in class

"Success!"

'Success!'

<font color="8888CC">Implement a small function `get_latest_block()` that returns the last block of the blockchain. This function is a luxury that will make the code easier to understand for the reader.</font>

> <details><summary><strong>Click to display help</strong></summary>Do you really need help for a function that is just 1 line long? ;)</details>

In [147]:
dummyBlockchain.get_latest_block().build_payload()

{'issuerPublicKey': '000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 'timestamp': 1736945827169,
 'indexInBlockchain': 0,
 'parentBlockHash': '0000000000000000000000000000000000000000000000000000000000000000',
 'certificateList': []}

In [151]:
dummyBlockchain.get_latest_block().is_legit()

True

### Blockchain Corruption

Continuing with the goal of helping you realize the **inherent security of the blockchain**, the `blockList` field of the `Blockchain` class has been **intentionally made public** so that you can **completely modify it**. Therefore, I will not ask you to implement functions for adding or removing blocks (removal being forbidden anyway).

Following this logic, it is still necessary to be able to **verify that a blockchain has not been corrupted**, whether by the addition of bad blocks or the alteration of data in older blocks.

***

<font color="8888CC">In the `Certificate`, `Block`, and `Blockchain` classes, implement the function `is_legit()` that returns `True` if the object in question is legal and `False` if it is corrupted. **I expect these functions to test all possibilities of data corruption.**

Some examples of data corruption:
* The signature is not valid
* The blocks are poorly indexed
* etc...

Find the others!</font>

> The following validation cell shows you the classic process of using the blockchain.

> <details><summary><strong>Click to display help for </strong><code>Certificate</code></summary>With your implementation of the `Certificate` class, you don't have many things to test. Just ensure that the signature is valid.</details>

> <details><summary><strong>Click to display help for </strong><code>Block</code></summary>Remember that a block is a certificate that contains a list of certificates, nothing more.</details>

> <details><summary><strong>Click to display help for </strong><code>Blockchain</code></summary>This is the biggest part, as you need to ensure the validity of all the blocks as well as that the blocks are properly chained and indexed. Don't forget to treat the genesis block differently.</details>

In [152]:
walletAlice = Wallet()
publicKeyAlice = walletAlice.publicKey

certificateAlice1 = Certificate(publicKeyAlice)
walletAlice.sign(certificateAlice1)

certificateAlice2 = Certificate(publicKeyAlice)
walletAlice.sign(certificateAlice2)

blockchain = Blockchain()

latestBlock = blockchain.get_latest_block()
block1 = Block(publicKeyAlice, latestBlock.indexInBlockchain + 1, latestBlock.hash(), [certificateAlice1])
walletAlice.sign(block1)
blockchain.blockList.append(block1)

latestBlock = blockchain.get_latest_block()
block2 = Block(publicKeyAlice, latestBlock.indexInBlockchain + 1, latestBlock.hash(), [certificateAlice2])
walletAlice.sign(block2)
blockchain.blockList.append(block2)

assert blockchain.is_legit()

certificateAlice1.timestamp -= 1

assert not blockchain.is_legit()

"Assignment completed! Congratz!"

'Assignment completed! Congratz!'

***
***

# Bonus

The following two exercises are only bonus points for this assignment. If you choose not to do them, it will have no impact on your future work.

If you attempt them but everything is incorrect, I will not take it into consideration at all, so feel free to give them a try (if anything, it might be worth it).

### Bonus 1: Readable Blockchain Display

For the 3rd assignment, you will likely need to regularly check or debug the data in your blockchain. To make your life easier, we will write some functions to display the entire blockchain in a readable way.

<font color="8888CC">For each of the classes `Wallet`, `Certificate`, `Block`, and `Blockchain`, implement the `display()` function. It should provide you with a visual representation of all the important information of the respective object. I won't impose any specific format; you can do it your way.</font>

> Don't hesitate to have them call each other when possible to save time.

> You can use the `dumps` function from the `json` package, or simply return something that is not a `string` because Jupyter Notebook formats dictionaries very nicely...

In [154]:
walletAlice.display()

Public key: 30820122300d06092a864886f70d01010105000382010f003082010a0282010100c4b8960db6a88207d04c08989cb0613ce7f72f5b354e59ca39fc2f750ec494dac285c2c411b4277ea5f362b7ad526d018091eedc89794d90883a655b668642c8a45f9786bc5daae76face252c8ada6902e620fdecd1a6eb3fe9848d924856f628e5894f33bafe282ff8c7943984fc0af639068555bfa6ff6b0238d9c4689f1250b2792c3b78f132a53e9374c83d4b0b76e846b0e5dc7a538db5bbcf4795d452122fc62149efc7afb3a71be9b71736f1786653bcbfa41b3ed7b517f6795508d6413924a90b3c7c4a44c30f6913edb07b8dffef96fe229f619c16f25fb340ea5b43dab94529e0211ba96d1621a7df2cd404c9025d4e3782c202230c7040b55bad90203010001
Private key: 308204a20201000282010100c4b8960db6a88207d04c08989cb0613ce7f72f5b354e59ca39fc2f750ec494dac285c2c411b4277ea5f362b7ad526d018091eedc89794d90883a655b668642c8a45f9786bc5daae76face252c8ada6902e620fdecd1a6eb3fe9848d924856f628e5894f33bafe282ff8c7943984fc0af639068555bfa6ff6b0238d9c4689f1250b2792c3b78f132a53e9374c83d4b0b76e846b0e5dc7a538db5bbcf4795d452122fc62149efc7afb3a71be9b71736f1786653bcbfa

In [156]:
blockchain.display()

Issuer public key: 000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Signature: None
Timestamp: 1736946169389
Hash: cdd0955c2fba3893f1ac7380a8ba59d45eae207f51d6f6b2677bcf31007e0588
Is legit: True


Issuer public key: 30820122300d06092a864886f70d01010105000382010f003082010a0282010100c4b8960db6a88207d04c08989cb0613ce7f72f5b354e59ca39fc2f750ec494dac285c2c411b4277ea5f362b7ad526d018091eedc89794d90883a655b668642c8a45f9786bc5daae76face252c8ada6902e620fdecd1a6eb3fe

[autoreload of block failed: Traceback (most recent call last):
  File "/home/bafbi/school/blockchain/Assignments_2/venv/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/bafbi/school/blockchain/Assignments_2/venv/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "/home/bafbi/school/blockchain/Assignments_2/venv/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
    ~~~~~~^^^^^^
  File "/home/bafbi/school/blockchain/Assignments_2/venv/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "/home/bafbi/school/blockchain/Assignments_2/venv/lib/python3.13/sit

### Bonus 2: TD 2 Report

To properly conclude this 2nd assignment and to help you understand why such a data structure is impossible to modify without causing complete corruption of the blockchain, I invite you to "mistreat" your implementation. Create situations of corruption that you find interesting and test how your blockchain reacts in these situations.

**Imagine having to sell such technology to a major client. You must convince them, through a written report in the form of a Notebook, and with concrete examples, that your technology is unbreakable.**

<font color="8888CC">Create a new Notebook for this report. To do this, in the main tab of Jupyter Notebook, in the upper right corner, there is a ```New``` button. Click on it and select the ```Python 3``` kernel.</font>

> I recommend generating 2 wallets to get interesting situations. Also, draw inspiration from what we have seen in class.

> If you want to format the text you include in this Notebook, I suggest you take inspiration from the texts in this Notebook by double-clicking on one of the texts to see the source code. At the top of the Notebook, you can change the type of a cell by replacing ```Code``` with ```Markdown```.

In [1]:
# Scenario 1: Tampering with a Certificate's Data
walletBob = Wallet()
publicKeyBob = walletBob.publicKey

certificateBob = Certificate(publicKeyBob)
walletBob.sign(certificateBob)

blockchain = Blockchain()

latestBlock = blockchain.get_latest_block()
block = Block(publicKeyBob, latestBlock.indexInBlockchain + 1, latestBlock.hash(), [certificateBob])
walletBob.sign(block)
blockchain.blockList.append(block)

# Tamper with the certificate's data
certificateBob.timestamp -= 1

print("Scenario 1: Tampering with a Certificate's Data")
print("Is the blockchain legit?", blockchain.is_legit())

# Scenario 2: Tampering with a Block's Data
certificateBob2 = Certificate(publicKeyBob)
walletBob.sign(certificateBob2)

latestBlock = blockchain.get_latest_block()
block2 = Block(publicKeyBob, latestBlock.indexInBlockchain + 1, latestBlock.hash(), [certificateBob2])
walletBob.sign(block2)
blockchain.blockList.append(block2)

# Tamper with the block's data
block2.indexInBlockchain += 1

print("\nScenario 2: Tampering with a Block's Data")
print("Is the blockchain legit?", blockchain.is_legit())

# Scenario 3: Tampering with the Blockchain's Structure
blockchain.blockList.pop()  # Remove the last block

print("\nScenario 3: Tampering with the Blockchain's Structure")
print("Is the blockchain legit?", blockchain.is_legit())

NameError: name 'Wallet' is not defined